# Computing the matrix spectrum and solving linear systems of differential equations

We are given the following matrix:

$A = \begin{bmatrix}
1 & 2 & 3 \\
2 & 1 & 4 \\
3 & 4 & 1
\end{bmatrix}$

Tasks:
1.   Compute the spectrum (eigenvalues and eigenvectors)
2.   Solve linear systems of differential equations of the form:

$\dot{\mathbf{x}} = A \mathbf{x}, \quad \mathbf{x} =
\begin{bmatrix} x_1(t) \\ x_2(t) \end{bmatrix}$

and find the general solution.

## 1. Compute the spectrum

In the following block the spectrum (eigenvalues and eigenvectors) is computed using NumPy’s built-in functions.

In [3]:
import numpy as np

A = np.array([
    [1, 2, 3],
    [2, 1, 4],
    [3, 4, 1]
])

eigenvalues, eigenvectors = np.linalg.eig(A)

print("Eigenvalues:")
print(eigenvalues)

print("\nEigenvectors (column i = eigenvector for eigenvalue i):")
print(eigenvectors)

Eigenvalues:
[ 7.07467358 -0.88679099 -3.1878826 ]

Eigenvectors (column i = eigenvector for eigenvalue i):
[[ 0.50578521  0.82403773 -0.25523155]
 [ 0.58437383 -0.54492509 -0.60130182]
 [ 0.63457746 -0.15497893  0.75716113]]


Computing the spectrum from scratch using QR iteration.

In [5]:
import sympy as sp

# Basic utility functions
def my_eye(n):
    result = np.zeros((n,n), dtype=np.float64)
    for i in range(n):
        result[i][i] = 1.0
    return result

def my_copy(v):
    n = len(v)
    result = np.zeros(n, dtype=np.float64)
    for i in range(n):
        result[i] = v[i]
    return result

def is_upper_tri(A, tol):
    n = len(A)
    for i in range(n):
        for j in range(i):
            if np.abs(A[i][j]) > tol:
                return False
    return True

# QR decomposition (Householder)
def my_qr(A, standard=False):
    m, n = A.shape
    Q = np.eye(m)
    R = np.copy(A)
    end = n-1 if m == n else n

    for i in range(end):
        H = np.eye(m)
        a = R[i:, i]
        norm_a = np.linalg.norm(a)
        if a[0] < 0.0:
            norm_a = -norm_a
        v = a / (a[0] + norm_a)
        v[0] = 1.0
        h = np.eye(len(a)) - (2 / np.dot(v, v)) * np.outer(v, v)
        H[i:, i:] = h
        Q = np.matmul(Q, H)
        R = np.matmul(H, R)

    if standard:
        S = np.zeros((n,n))
        for i in range(n):
            S[i][i] = -1.0 if R[i][i] < 0.0 else 1.0
        Q = np.matmul(Q, S)
        R = np.matmul(S, R)

    return Q, R

# Eigenvalues and eigenvectors via QR iteration
def my_eigen(A):
    n = len(A)
    X = np.copy(A)
    pq = np.eye(n)
    max_ct = 10000
    ct = 0

    while ct < max_ct:
        Q, R = my_qr(X)
        pq = np.matmul(pq, Q)
        X = np.matmul(R, Q)
        ct += 1
        if is_upper_tri(X, 1.0e-8):
            break
    else:
        print("WARN: no converge ")

    e_vals = np.zeros(n, dtype=np.float64)
    for i in range(n):
        e_vals[i] = X[i][i]
    e_vecs = np.copy(pq)
    return e_vals, e_vecs

A = np.array([[1, 2, 3],
              [2, 1, 4],
              [3, 4, 1]], dtype=np.float64)

print("Matrix A: ")
print(A)

# Characteristic polynomial
lam = sp.symbols('lambda')
A_sym = sp.Matrix(A)
char_poly = (A_sym - lam*sp.eye(3)).det()
print("\nCharacteristic equation |A - λI| = 0:")
poly_str = sp.pretty(char_poly)
print(poly_str + " = 0")

# Eigenvalues and eigenvectors
e_vals, e_vecs = my_eigen(A)

print("\nEigenvalues (from QR iteration):")
for i, val in enumerate(e_vals, start=1):
    print(f"λ{i} = {val}")

print("\nEigenvectors:")
for i in range(e_vecs.shape[1]):
    print(f"v{i+1} = {e_vecs[:, i]}")

eigenvector_matrix = np.column_stack([e_vecs[:, i] for i in range(e_vecs.shape[1])])
print("\nMatrix with eigenvectors as columns (v1, v2, v3):")
print(eigenvector_matrix)

Matrix A: 
[[1. 2. 3.]
 [2. 1. 4.]
 [3. 4. 1.]]

Characteristic equation |A - λI| = 0:
   3        2                
- λ  + 3.0⋅λ  + 26.0⋅λ + 20.0 = 0

Eigenvalues (from QR iteration):
λ1 = 7.07467358251512
λ2 = -3.1878825962647523
λ3 = -0.8867909862503733

Eigenvectors:
v1 = [0.50578521 0.58437383 0.63457746]
v2 = [ 0.25523155  0.60130182 -0.75716113]
v3 = [-0.82403773  0.54492509  0.15497893]

Matrix with eigenvectors as columns (v1, v2, v3):
[[ 0.50578521  0.25523155 -0.82403773]
 [ 0.58437383  0.60130182  0.54492509]
 [ 0.63457746 -0.75716113  0.15497893]]


## 2. Solve linear systems of differential equations

We consider systems of the form:

$\dot{\mathbf{x}} = A \mathbf{x}, \quad \mathbf{x}(t) = \begin{bmatrix} x_1(t) \\ x_2(t) \end{bmatrix}$

Example systems
\begin{cases}
\dot{x}_1 = 2 x_1 + x_2 \\
\dot{x}_2 = 3 x_1 + 4 x_2
\end{cases}

\begin{cases}
\dot{x}_1 = 2 x_1 - 6 x_2 \\
\dot{x}_2 = 3 x_1 + 4 x_2
\end{cases}


In [9]:
import sympy as sp

def solve_linear_system(A):
    t = sp.symbols('t')
    n = A.shape[0]
    X = sp.Matrix([sp.Function(f"x{i+1}")(t) for i in range(n)])
    system = X.diff(t) - A * X
    sol = sp.dsolve(system)

    print("\nGeneral solution for system X' = A X:")
    for eq in sol:
        sp.pprint(eq)
    print("\n" + "-"*50 + "\n")

    return None

A1 = sp.Matrix([
    [2, 1],
    [3, 4]
])

A2 = sp.Matrix([
    [2, -6],
    [3,  4]
])

solve_linear_system(A1)
solve_linear_system(A2)


General solution for system X' = A X:
                      5⋅t
              t   C₂⋅ℯ   
x₁(t) = - C₁⋅ℯ  + ───────
                     3   
            t       5⋅t
x₂(t) = C₁⋅ℯ  + C₂⋅ℯ   

--------------------------------------------------


General solution for system X' = A X:
          ⎛C₁   √17⋅C₂⎞  3⋅t              ⎛√17⋅C₁   C₂⎞  3⋅t           
x₁(t) = - ⎜── + ──────⎟⋅ℯ   ⋅cos(√17⋅t) - ⎜────── - ──⎟⋅ℯ   ⋅sin(√17⋅t)
          ⎝3      3   ⎠                   ⎝  3      3 ⎠                
            3⋅t                  3⋅t           
x₂(t) = C₁⋅ℯ   ⋅cos(√17⋅t) - C₂⋅ℯ   ⋅sin(√17⋅t)

--------------------------------------------------




\begin{aligned}
\begin{cases}
\dot{x}_1 = x_1 \\
\dot{x}_2 = 6 x_2 - 7 x_3 + 3 x_4\\
\dot{x}_3 = 3 x_3 - x_4\\
\dot{x}_4 = -4 x_2 + 9 x_3 - 3x_4
\end{cases}
\qquad
\begin{cases}
\dot{x}_1 = -2 x_1 - 6 x_2 \\
\dot{x}_2 = 3 x_1 + 4 x_2
\end{cases}
\qquad
\begin{cases}
\dot{x}_1 = 3 x_1 - 4 x_2 - x_3 \\
\dot{x}_2 = - x_2 - x_3\\
\dot{x}_3 = -4 x_2 - 2 x_3
\end{cases}
\end{aligned}


In [11]:
A3 = sp.Matrix([
    [1, 0, 0, 0],
    [0, 6, -7, 3],
    [0, 0, 3, -1],
    [0, -4, 9, -3]
])

A4 = sp.Matrix([
    [-2, -6],
    [3,  4]
])

A5 = sp.Matrix([
    [3, -4, -1],
    [0, -1, -1],
    [0, -4, 2]
])

solve_linear_system(A3)
solve_linear_system(A4)
solve_linear_system(A5)


General solution for system X' = A X:
            t
x₁(t) = C₁⋅ℯ 
              2  2⋅t                    2⋅t                       2⋅t
x₂(t) = 2⋅C₂⋅t ⋅ℯ    + t⋅(4⋅C₂ + 4⋅C₄)⋅ℯ    + (C₂ + 4⋅C₃ + 4⋅C₄)⋅ℯ   
              2  2⋅t         2⋅t           2⋅t
x₃(t) = 2⋅C₂⋅t ⋅ℯ    + 4⋅C₃⋅ℯ    + 4⋅C₄⋅t⋅ℯ   
              2  2⋅t                    2⋅t                  2⋅t
x₄(t) = 2⋅C₂⋅t ⋅ℯ    - t⋅(4⋅C₂ - 4⋅C₄)⋅ℯ    + (4⋅C₃ - 4⋅C₄)⋅ℯ   

--------------------------------------------------


General solution for system X' = A X:
                     t                       t         
x₁(t) = - (C₁ - C₂)⋅ℯ ⋅sin(3⋅t) - (C₁ + C₂)⋅ℯ ⋅cos(3⋅t)
            t                t         
x₂(t) = C₁⋅ℯ ⋅cos(3⋅t) - C₂⋅ℯ ⋅sin(3⋅t)

--------------------------------------------------


General solution for system X' = A X:
            -2⋅t       3⋅t
x₁(t) = C₁⋅ℯ     + C₂⋅ℯ   
                       3⋅t
            -2⋅t   C₃⋅ℯ   
x₂(t) = C₁⋅ℯ     - ───────
                      4   
            -2⋅t       3⋅t
x₃(t